In [1]:
import sys, os
sys.path.insert(0, os.path.join(".."))

In [2]:
import pdfimage.pdf_read

In [3]:
pdf = pdfimage.pdf_read.PDF(os.path.join("..", "PDFReference.pdf"))
pdf.version

'%PDF-1.4'

In [4]:
pdf.trailer

PDFDictionary({PDFName(b'Size'): PDFNumeric(31667), PDFName(b'ID'): PDFArray([PDFString(b"\x19WE'\xec\xbf\x00\xe3\xec\x03s\x87\x983\xee\xf6"), PDFString(b'$\xee\x9e\xdb}\xe4\r\xb8b\xfd\xb4\xc5\xd3I5\x85')]), PDFName(b'Info'): PDFObjectId(7, 0), PDFName(b'Root'): PDFObjectId(1, 0), PDFName(b'Encrypt'): PDFObjectId(31666, 0)})

The XREF table of objects and their locations is available as a dictionary.

In [6]:
list(pdf.object_lookup.items())[:5]

[(PDFObjectId(1, 0), 15),
 (PDFObjectId(2, 0), 670),
 (PDFObjectId(3, 0), 791),
 (PDFObjectId(4, 0), 2566),
 (PDFObjectId(5, 0), 2624)]

Check that for each object is either a single object, or a pair of objects where the second object is a placeholder for a PDF Stream.  If a stream, then load it.

In [7]:
for key, loc in pdf.object_lookup.items():
    items = pdf.object_at(loc)
    if len(items) == 1:
        continue
    if len(items) == 2 and isinstance(items[1], pdfimage.pdf_read.PDFStreamIndicator):
        obj = pdf.full_object_at(loc)
        continue
    raise AssertionError()

Example of a simple object

In [9]:
pdf.object_at(15)

[PDFDictionary({PDFName(b'Type'): PDFName(b'Catalog'), PDFName(b'Pages'): PDFObjectId(2, 0), PDFName(b'Metadata'): PDFObjectId(3, 0), PDFName(b'Outlines'): PDFObjectId(4, 0), PDFName(b'PageLabels'): PDFObjectId(5, 0), PDFName(b'OpenAction'): PDFObjectId(6, 0), PDFName(b'PageMode'): PDFName(b'UseOutlines'), PDFName(b'ViewerPreferences'): PDFDictionary({PDFName(b'DisplayDocTitle'): PDFBoolean(True)})})]

Example of an object with stream.  The `full_object_at` method attempts to recursingly read all the objects which are references.  We do not attempt to deal with cyclic references, so this can lead to a recursion error

In [17]:
obj = pdf.full_object_at(791)
obj

PDFDictionary({PDFName(b'Type'): PDFName(b'Metadata'), PDFName(b'Subtype'): PDFName(b'XML'), PDFName(b'Length'): PDFNumeric(1691)}, stream length=1691)

In [18]:
obj.stream_contents[:10]

b'\xcd\x14\xedob\r\xdb$\xc8e'

Trying to load all of the following object would lead to a recursion error.

In [19]:
pdf.object_at(670)

[PDFDictionary({PDFName(b'Type'): PDFName(b'Pages'), PDFName(b'Kids'): PDFArray([PDFObjectId(8, 0), PDFObjectId(9, 0), PDFObjectId(10, 0), PDFObjectId(11, 0), PDFObjectId(12, 0), PDFObjectId(13, 0), PDFObjectId(14, 0), PDFObjectId(15, 0), PDFObjectId(16, 0), PDFObjectId(17, 0)]), PDFName(b'Count'): PDFNumeric(978)})]